In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
X = data.iloc[:,0:12]
y = data.iloc[:,12]
data.head()

In [ ]:
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
cor_target = abs(cor["DEATH_EVENT"])#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.07]
relevant_features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=None)

In [ ]:
classifiers = [LogisticRegression, MLPClassifier, KNeighborsClassifier, tree.DecisionTreeClassifier, RandomForestClassifier, SVC]
for model in classifiers:
    classifier = model()
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test) 
    print("{} model accuracy: {}".format(model, np.amax(cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10), axis = 0)))

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X_test)

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(X_train.iloc[:50])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_test.iloc[:50])